In [384]:
from dolfinx import default_scalar_type
from dolfinx import fem
from dolfinx.fem.petsc import LinearProblem
from dolfinx.io import XDMFFile
from dolfinx.mesh import locate_entities, create_unit_square, meshtags
from dolfinx.plot import vtk_mesh


from mpi4py import MPI

import ufl

import numpy as np
import pyvista

mesh = create_unit_square(MPI.COMM_WORLD, 50, 50)

In [385]:
u_ex = lambda x: 1+x[1]**3-2*x[0]-x[0]**2
x = ufl.SpatialCoordinate(mesh)

f = 4+2/x[0]-6*x[1]
n = ufl.FacetNormal(mesh)
g = -ufl.dot(n, ufl.grad(u_ex(x)))

V = fem.functionspace(mesh, ("Lagrange", 1))
u, v = ufl.TrialFunction(V), ufl.TestFunction(V)
F = ufl.dot(ufl.grad(u), ufl.grad(v)) *x[0]* ufl.dx - f*v*x[0] * ufl.dx

In [386]:
boundaries = [(1, lambda x: np.isclose(x[0], 0)),
              (2, lambda x: np.isclose(x[0], 1)),
              (3, lambda x: np.isclose(x[1], 0)),
              (4, lambda x: np.isclose(x[1], 1))]

In [387]:
facet_indices, facet_markers = [], []
fdim = mesh.topology.dim - 1
for (marker, locator) in boundaries:
    facets = locate_entities(mesh, fdim, locator)
    facet_indices.append(facets)
    facet_markers.append(np.full_like(facets, marker))
facet_indices = np.hstack(facet_indices).astype(np.int32)
facet_markers = np.hstack(facet_markers).astype(np.int32)
sorted_facets = np.argsort(facet_indices)
facet_tag = meshtags(mesh, fdim, facet_indices[sorted_facets], facet_markers[sorted_facets])

In [388]:
mesh.topology.create_connectivity(mesh.topology.dim-1, mesh.topology.dim)
with XDMFFile(mesh.comm, "facet_tags.xdmf", "w") as xdmf:
    xdmf.write_mesh(mesh)
    xdmf.write_meshtags(facet_tag, mesh.geometry)

In [389]:
ds = ufl.Measure("ds", domain=mesh, subdomain_data=facet_tag)

In [390]:
class BoundaryCondition():
    def __init__(self, type, marker, values):
        self._type = type
        if type == "Dirichlet":
            u_D = fem.Function(V)
            u_D.interpolate(values)
            facets = facet_tag.find(marker)
            dofs = fem.locate_dofs_topological(V, fdim, facets)
            self._bc = fem.dirichletbc(u_D, dofs)
        elif type == "Neumann":
            self._bc = ufl.inner(values, v)*x[0]* ds(marker)
        else:
            raise TypeError("Unknown boundary condition: {0:s}".format(type))
    @property
    def bc(self):
        return self._bc

    @property
    def type(self):
        return self._type

# Define the Dirichlet condition
boundary_conditions = [BoundaryCondition("Neumann", 1, g),
                       BoundaryCondition("Dirichlet", 2, u_ex),
                       BoundaryCondition("Neumann", 3, g),
                       BoundaryCondition("Neumann", 4, g)]


In [391]:
bcs = []
for condition in boundary_conditions:
    if condition.type == "Dirichlet":
        bcs.append(condition.bc)
    else:
        F += condition.bc

In [392]:
# Solve linear variational problem
a = ufl.lhs(F)
L = ufl.rhs(F)
problem = LinearProblem(a, L, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

# Visualize solution
pyvista_cells, cell_types, geometry = vtk_mesh(V)
grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, geometry)
grid.point_data["u"] = uh.x.array
grid.set_active_scalars("u")

plotter = pyvista.Plotter()
plotter.add_text("uh", position="upper_edge", font_size=14, color="black")
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("neumann_dirichlet.png")

Widget(value='<iframe src="http://localhost:45345/index.html?ui=P_0x7fd7474879d0_36&reconnect=auto" class="pyv…

In [393]:
# Compute L2 error and error at nodes
V_ex = fem.functionspace(mesh, ("Lagrange", 2))
u_exact = fem.Function(V_ex)
u_exact.interpolate(u_ex)
error_L2 = np.sqrt(mesh.comm.allreduce(fem.assemble_scalar(fem.form((uh - u_exact)**2 * ufl.dx)), op=MPI.SUM))

u_vertex_values = uh.x.array
uex_1 = fem.Function(V)
uex_1.interpolate(u_ex)
u_ex_vertex_values = uex_1.x.array
error_max = np.max(np.abs(u_vertex_values - u_ex_vertex_values))
error_max = mesh.comm.allreduce(error_max, op=MPI.MAX)
print(f"Error_L2 : {error_L2:.2e}")
print(f"Error_max : {error_max:.2e}")

Error_L2 : 1.58e-04
Error_max : 9.96e-04
